In [1]:
%load_ext memory_profiler

In [3]:
from argo_utils.argo_utils import *

# convert_conditions_to_argo_string

In [5]:
_range = range(0, 100)

X_list = [X for i in _range]

col_name = [(f'int_col{i}', f'ohe_cat_col{i}', f'float_col{i}')for i in _range]

col_name = [c for t in col_name for c in t]

X_big = pd.concat(X_list, axis=1)

In [56]:
%%file test.py

import pandas as pd
import numpy as np

@profile
def my_func():
    l = [1] * 10**6
#     l_ = l * 2
    l = l * 2
    return l
#     X = pd.DataFrame({
#         'int_col': np.random.randint(0, 10, 1000000),
#         'ohe_cat_col': np.random.randint(0, 2, 1000000),
#         'float_col': np.random.uniform(0, 1, 1000000)
#     })
    
#     int_mask = (X - X.round()).sum() == 0
#     int_cols = list(X.columns[int_mask])
#     float_cols = list(X.columns[~int_mask])
#     ohe_cat_cols = []
#     for col in int_cols:
#         unique_values = X[col].unique()
#         unique_values.sort()
#         unique_values = set(unique_values)
#         if unique_values == {0, 1}:
#             ohe_cat_cols.append(col)
#     return int_cols, ohe_cat_cols, float_cols

if __name__ == '__main__':
    my_func()

Overwriting test.py


In [57]:
!python -m memory_profiler test.py

Filename: test.py

Line #    Mem usage    Increment  Occurences   Line Contents
     5   67.699 MiB   67.699 MiB           1   @profile
     6                                         def my_func():
     7   75.332 MiB    7.633 MiB           1       l = [1] * 10**6
     8                                         #     l_ = l * 2
     9   90.594 MiB   15.262 MiB           1       l = l * 2
    10   90.594 MiB    0.000 MiB           1       return l




In [58]:
l = [
    ('col_cat', '<=', 0.5), ('col_cat', '>', 0.5), ('col_float', '>=', 1.1), ('col_int', '>=', 1.2)
]

In [59]:
list_of_conditions = l * 10

In [60]:
columns_int = ['col_int']
columns_cat = ['col_cat']

In [61]:
%memit convert_conditions_to_argo_string(list_of_conditions, columns_int, columns_cat)

peak memory: 123.23 MiB, increment: -0.18 MiB


In [71]:
%memit [i for i in range(0, 10000000)]

peak memory: 411.04 MiB, increment: 282.73 MiB


In [78]:
np.random.seed(0)
X = pd.DataFrame({
    'A': np.random.randint(0, 2, 1000000),
    'B': np.random.randint(0, 2, 1000000)
})
y = pd.Series(np.random.randint(0, 2, 1000000))

In [79]:
%memit return_binary_pred_perf_of_set_numpy(y_true=y, y_preds=X, y_preds_columns=X.columns)

peak memory: 195.46 MiB, increment: 30.52 MiB


In [76]:
%memit [i for i in range(0, 10)]

peak memory: 134.41 MiB, increment: 0.00 MiB


In [86]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

In [87]:
%memit sum_of_lists(1000000)

peak memory: 186.46 MiB, increment: 27.93 MiB


In [22]:
%mprun -f convert_conditions_to_argo_string convert_conditions_to_argo_string(list_of_conditions, columns_int, columns_cat)

Filename: /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/argo_utils/argo_utils.py

Line #    Mem usage    Increment  Occurences   Line Contents
    10    277.0 MiB    277.0 MiB           1   def convert_conditions_to_argo_string(list_of_conditions: list, columns_int: list,
    11                                                                               columns_cat: list) -> str:
    12                                             """
    13                                             Converts a list of conditions to the standard ARGO string format.
    14                                         
    15                                             Args:
    16                                                 list_of_conditions (list): Each element contains a tuple of the feature (str),
    17                                                     operator (str) and value (numeric) for each condition in the rule.
    18                                                 column

In [11]:
%mprun con(list_of_conditions, columns_int, columns_cat)

In [4]:
import memory_profiler

In [6]:
%%writefile test.py
@profile
def test():
    a = [1] * 1000000
    b = [2] * 9000000
    del b
    return a

Writing test.py


In [7]:
from test import test

NameError: name 'profile' is not defined

In [52]:
%mprun -T mprof0 -f test test()



*** Profile printout saved to text file mprof0. 


Filename: /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/tests/development/test.py

Line #    Mem usage    Increment  Occurences   Line Contents
     1    276.3 MiB    276.3 MiB           1   def test():
     2    276.3 MiB      0.0 MiB       10003       a = [1] * 1000000
     3    276.3 MiB      0.0 MiB           1       b = [2] * 9000000
     4    276.3 MiB      0.0 MiB           1       del b
     5                                             return a

In [53]:
print(open('mprof0', 'r').read())

Filename: /Users/jlaidler/Documents/tigress/tigress/argo/argo/argo_utils/tests/development/test.py

Line #    Mem usage    Increment  Occurences   Line Contents
     1    276.3 MiB    276.3 MiB           1   def test():
     2    276.3 MiB      0.0 MiB       10003       a = [1] * 1000000
     3    276.3 MiB      0.0 MiB           1       b = [2] * 9000000
     4    276.3 MiB      0.0 MiB           1       del b
     5                                             return a
